In [1]:
import re

str = u'陈奕迅演唱(十年)、(浮夸)、(不要说话)'
string = re.finditer('\((.*?)\)', str)
print(string)

In [41]:
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

In [42]:
from fastai import *
from fastai.vision import *
from fastai.text import *
from fastai.callbacks import *

In [43]:
DATA_ROOT = Path("") 

train, test = [pd.read_csv(DATA_ROOT / fname) for fname in ["task2_trainset.csv", "task2_public_testset.csv"]]

In [44]:
train.drop('Title',axis=1,inplace=True)
train.drop('Categories',axis=1,inplace=True)
test.drop('Title',axis=1,inplace=True)
test.drop('Categories',axis=1,inplace=True)
####################################################
train.drop('Created Date',axis=1, inplace=True)
train.drop('Authors',axis=1,inplace=True)
test.drop('Created Date',axis=1, inplace=True)
test.drop('Authors',axis=1,inplace=True)

In [45]:
train['THEORETICAL']=0
train['ENGINEERING']=0
train['EMPIRICAL']=0
train['OTHERS']=0
for i in range(train.Id.count()):
    string = train['Task 2'][i]
    train['THEORETICAL'][i] = 1 if (string.find('THEORETICAL')!=-1) else 0
    train['ENGINEERING'][i] = 1 if (string.find('ENGINEERING')!=-1) else 0
    train['EMPIRICAL'][i] = 1 if (string.find('EMPIRICAL')!=-1) else 0
    train['OTHERS'][i] = 1 if (string.find('OTHERS')!=-1) else 0   
train.head()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:

,Id,Abstract,Task 2,THEORETICAL,ENGINEERING,EMPIRICAL,OTHERS
0,D00001,Rapid popularity of Internet of Things (IoT) a...,THEORETICAL,1,0,0,0
1,D00002,"In this paper, we address the problem of compu...",THEORETICAL,1,0,0,0
2,D00003,High quality upsampling of sparse 3D point clo...,ENGINEERING,0,1,0,0
3,D00004,Internet is the main source of information now...,EMPIRICAL,0,0,1,0
4,D00005,Automated Facial Expression Recognition (FER) ...,ENGINEERING,0,1,0,0


In [46]:
train.drop('Task 2',axis=1,inplace=True)
train.head()

,Id,Abstract,THEORETICAL,ENGINEERING,EMPIRICAL,OTHERS
0,D00001,Rapid popularity of Internet of Things (IoT) a...,1,0,0,0
1,D00002,"In this paper, we address the problem of compu...",1,0,0,0
2,D00003,High quality upsampling of sparse 3D point clo...,0,1,0,0
3,D00004,Internet is the main source of information now...,0,0,1,0
4,D00005,Automated Facial Expression Recognition (FER) ...,0,1,0,0


In [47]:
print(train.THEORETICAL.value_counts()[1] / len(train))
print(train.ENGINEERING.value_counts()[1] / len(train))
print(train.EMPIRICAL.value_counts()[1] / len(train))
print(train.OTHERS.value_counts()[1] / len(train))

0.45971428571428574
0.48442857142857143
0.3057142857142857
0.037


In [48]:
max_sent = max([len(train['Abstract']) for data in train])
max_sent

7000

In [49]:
train['Abstract'][6094]

'This article reports on an exploratory case study conducted to examine the viability of Second Life (SL) as an environment for physical simulations and microworlds.$$$It begins by discussing specific features of the SL environment relevant to its use as a support for microworlds and simulations as well as a few differences found between SL and traditional simulators such as Modellus, along with their implications to simulations, as a support for subsequent analysis.$$$Afterwards, we will use Narayanasamy et al. and Johnston and Whitehead criteria to analyze the SL environment and determine into which of training simulators, games, simulation games, or serious games categories SL fits best.$$$We conclude that SL shows itself as a huge and sophisticated simulator of an entire Earthlike world used by thousands of users to simulate real life in some sense and a viable and flexible platform for microworlds and simulations.'

In [50]:
for i in range(train.Id.count()):
    train['Abstract'][i]=train['Abstract'][i].replace("$$$", " ")
    train['Abstract'][i]=train['Abstract'][i].replace(".", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("->", "to")
    train['Abstract'][i]=train['Abstract'][i].replace("~", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("&", "and")
    train['Abstract'][i]=train['Abstract'][i].replace(",", " ")
    train['Abstract'][i]=train['Abstract'][i].replace(":", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("%", "percentage")
    train['Abstract'][i]=train['Abstract'][i].replace("C++", "c plus plus")
    train['Abstract'][i]=train['Abstract'][i].replace("w.r.t.", "in respect of")
    train['Abstract'][i]=train['Abstract'][i].replace("e.g.", "example")
    train['Abstract'][i]=train['Abstract'][i].replace("etc.", "and so on")
    #train['Abstract'][i]=train['Abstract'][i].replace("'s'", "s")
    #train['Abstract'][i]=train['Abstract'][i].replace("%", "in other word")
    train['Abstract'][i]=train['Abstract'][i].replace("(A)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(B)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(C)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(D)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(a)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(b)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(c)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(d)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(e)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(1)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(2)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(3)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(4)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(5)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(i)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(ii)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(iii)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(iv)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("i)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("ii)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("iii)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("iv)", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("(", " ")
    train['Abstract'][i]=train['Abstract'][i].replace(")", " ")
    

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is tr

In [51]:
train['Abstract'][6169]

"Euphonic conjunctions  sandhis  form a very important aspect of Sanskrit morphology and phonology  The traditional and modern methods of studying about euphonic conjunctions in Sanskrit follow different methodologies  The former involves a rigorous study of the Paninian system embodied in Panini's Ashtadhyayi  while the latter usually involves the study of a few important sandhi rules with the use of examples  The former is not suitable for beginners  and the latter  not sufficient to gain a comprehensive understanding of the operation of sandhi rules  This is so since there are not only numerous sandhi rules and exceptions  but also complex precedence rules involved  The need for a new ontology for sandhi-tutoring was hence felt  This work presents a comprehensive ontology designed to enable a student-user to learn in stages all about euphonic conjunctions and the relevant aphorisms of Sanskrit grammar and to test and evaluate the progress of the student-user  The ontology forms the 

In [52]:
for i in range(test.Id.count()):
    test['Abstract'][i]=test['Abstract'][i].replace("$$$", " ")
    test['Abstract'][i]=test['Abstract'][i].replace(".", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("->", "to")
    test['Abstract'][i]=test['Abstract'][i].replace("~", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("&", "and")
    test['Abstract'][i]=test['Abstract'][i].replace(",", " ")
    test['Abstract'][i]=test['Abstract'][i].replace(":", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("%", "percentage")
    test['Abstract'][i]=test['Abstract'][i].replace("C++", "c plus plus")
    test['Abstract'][i]=test['Abstract'][i].replace("w.r.t.", "in respect of")
    test['Abstract'][i]=test['Abstract'][i].replace("e.g.", "example")
    test['Abstract'][i]=test['Abstract'][i].replace("etc.", "and so on")
    #test['Abstract'][i]=test['Abstract'][i].replace("'s'", "s")
    #test['Abstract'][i]=test['Abstract'][i].replace("%", "in other word")
    test['Abstract'][i]=test['Abstract'][i].replace("(A)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(B)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(C)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(D)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(a)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(b)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(c)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(d)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(e)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(1)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(2)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(3)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(4)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(5)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(i)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(ii)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(iii)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(iv)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("i)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("ii)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("iii)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("iv)", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("(", " ")
    test['Abstract'][i]=test['Abstract'][i].replace(")", " ")
    